In [5]:
from datetime import datetime
import pytz
import openpyxl


def get_data(output_file_path, input_sheet_name, start_row, end_row, start_timezone, sfd_timezone):
    try:
        workbook = openpyxl.load_workbook(output_file_path)
        
        sheet = workbook[input_sheet_name]

        cr_number_col = get_column_index(sheet, "CR number")
        collection_col = get_column_index(sheet, "Collection")
        duration_col = get_column_index(sheet, "Duration")
        Start_Date_col = get_column_index(sheet, "Start Date")
        End_Date_col = get_column_index(sheet, "End Date")

        for row_num in range(start_row, end_row + 1):

            # Skip rows containing delimter of table
            if sheet.cell(row=row_num, column=2).value == "CRs created/left:":
                continue
        
            cr_number = sheet.cell(row=row_num, column=cr_number_col).value
            collection = sheet.cell(row=row_num, column=collection_col).value
            duration = sheet.cell(row=row_num, column=duration_col).value

            start_date = sheet.cell(row=row_num, column=Start_Date_col).value
            start_date_object = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S")
            start_date_localized = start_timezone.localize(start_date_object)
            start_date_sfd = start_date_localized.astimezone(sfd_timezone)
            formatted_start_date = start_date_sfd.strftime("%#m/%#d/%Y %H:%M")

            end_date = sheet.cell(row=row_num, column=End_Date_col).value
            end_date_object = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S")
            end_date_localized = start_timezone.localize(end_date_object)
            end_date_sfd = end_date_localized.astimezone(sfd_timezone) 
            formatted_end_date = end_date_sfd.strftime("%#m/%#d/%Y %H:%M")
       
            fill_sfd(input_file_path, cr_number, collection, duration, formatted_start_date, formatted_end_date)

    except Exception as e:
        print(f"An error occurred: {e}")

def get_column_index(sheet, column_name):
    for col_num in range(1, sheet.max_column + 1):
        if sheet.cell(row=1, column=col_num).value == column_name:
            return col_num
    raise ValueError(f"Column '{column_name}' not found in the sheet")

def fill_sfd(input_file_path, cr_number, collection, duration, formatted_start_date, formatted_end_date):
    
    try:

        input_sheet_name = 'Scheduled for Deployment'

        # Read the existing data from the Excel file
        workbook = openpyxl.load_workbook(input_file_path)
        worksheet = workbook[input_sheet_name]

        # Get the column index for 'Group Name' and 'Change Request'
        group_name_col = None
        change_request_col = None
        sfd_duration_col = None
        planned_start_date_col = None
        planned_end_date_col  = None

        for sfd_column_idx in range(1, worksheet.max_column + 1):
            column_value = worksheet.cell(row=1, column=sfd_column_idx).value
            if column_value is not None:
                column_value_lower = column_value.lower()
                if column_value_lower == 'group name':
                    group_name_col = sfd_column_idx
                elif column_value_lower == 'change request':
                    change_request_col = sfd_column_idx
                elif column_value_lower =='duration':
                    sfd_duration_col = sfd_column_idx
                elif column_value_lower == 'planned start date':
                    planned_start_date_col = sfd_column_idx
                elif column_value_lower == 'planned end date':
                    planned_end_date_col = sfd_column_idx

        if group_name_col is None:
            raise ValueError("Error: 'Group name' column not found.")
        elif change_request_col is None:
            raise ValueError("Error: 'CR number' column not found.")
        elif planned_start_date_col is None:
            raise ValueError("Error: 'Planned Start Date' column not found.")
        elif planned_end_date_col  is None:
            raise ValueError("Error: 'Planned End Date' column not found.")

        collection_found = False

        # Update columns for the relevant rows
        for row_idx in range(2, worksheet.max_row + 1):
            group_name = worksheet.cell(row=row_idx, column=group_name_col).value
            if group_name and collection.lower() in group_name.lower():
                collection_found = True
                worksheet.cell(row=row_idx, column=change_request_col).value = cr_number
                worksheet.cell(row=row_idx, column=sfd_duration_col).value = duration
                worksheet.cell(row=row_idx, column=planned_start_date_col).value = formatted_start_date
                worksheet.cell(row=row_idx, column=planned_end_date_col).value = formatted_end_date

        if not collection_found:
            raise ValueError(f"Error: Collection '{collection}' not found.")

        # Save the changes
        workbook.save(input_file_path)
    
    except Exception as e:
        print(f"An error occured: {e}")

if __name__ == "__main__":
    
    start_row = int(input("Enter the row: "))
    end_row = int(input("Enter the row: "))

    start_timezone = pytz.timezone('Europe/London')
    sfd_timezone = pytz.timezone('America/Los_Angeles')

    #required sheet name
    input_sheet_name = 'April2024'
    #input_sheet_name = workbook.active

    links_file_path = 'C:/Users/pelyu/OneDrive/Рабочий стол/chg/git/linkspath.txt'

    with open(links_file_path, 'r', encoding='utf-8') as file:
    # Read the lines
        lines = file.readlines()

    output_file_path = lines[0].strip() if len(lines) > 0 else ''
    input_file_path = lines[1].strip() if len(lines) > 1 else '' 

    get_data(output_file_path, input_sheet_name, start_row, end_row, start_timezone, sfd_timezone)


An error occured: Error: Collection 'Patch Group - UB PROD' not found.
An error occured: Error: Collection 'Patch Group - United Bingdom - Group B PROD' not found.


Rewrites CR number - Duration - Start Date - End Date, we may add reboot supress column and week description